In [ ]:
# Установка необходимых библиотек
!pip install mido networkx matplotlib pandas

import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from mido import MidiFile
from google.colab import files
from collections import defaultdict

# Загрузка MIDI-файла
uploaded = files.upload()
midi_path = list(uploaded.keys())[0]

# Функция для анализа MIDI-файла и построения графа
def analyze_and_visualize_midi_graph(midi_path):
    # Загружаем MIDI-файл
    midi_file = MidiFile(midi_path)

    # Собираем события нот
    note_events = []
    for i, track in enumerate(midi_file.tracks):
        current_time = 0
        for msg in track:
            current_time += msg.time
            if msg.type == 'note_on' and msg.velocity > 0:  # Только "note_on" с положительной громкостью
                note_events.append(msg.note)

    # Построение графа с подсчётом всех переходов между нотами
    G = nx.DiGraph()
    transition_count = defaultdict(int)

    for i in range(len(note_events) - 1):
        current_note = note_events[i]
        next_note = note_events[i + 1]
        transition_count[(current_note, next_note)] += 1

    # Добавляем рёбра с весами в граф
    for (note1, note2), weight in transition_count.items():
        G.add_edge(note1, note2, weight=weight)

    # Визуализация графа
    plt.figure(figsize=(16, 12))  # Увеличенный размер для лучшей читаемости
    pos = nx.spring_layout(G, seed=42)  # Фиксированный layout для стабильного отображения

    # Цветовая схема для узлов и рёбер
    nx.draw_networkx_nodes(G, pos, node_color="lightblue", edgecolors="black", node_size=600, alpha=0.9)
    nx.draw_networkx_edges(G, pos, edge_color="cornflowerblue", width=1.5, alpha=0.6, arrows=True)
    nx.draw_networkx_labels(G, pos, font_size=10, font_family="sans-serif", font_weight="bold", font_color="darkblue")

    # Отображаем только рёбра с весом больше определенного порога
    important_edges = {(u, v): G[u][v]['weight'] for u, v in G.edges() if G[u][v]['weight'] > 1}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=important_edges, font_size=8, font_color="darkgreen")

    plt.title("MIDI Graph Visualization", fontsize=16, fontweight="bold", color="darkblue")
    plt.axis("off")
    plt.show()

    # Создание таблицы аналитических данных
    metrics = {
        'Note (MIDI)': [],
        'Degree': [],
        'In Degree': [],
        'Out Degree': [],
        'Average Edge Weight': []
    }

    for node in G.nodes():
        metrics['Note (MIDI)'].append(node)
        metrics['Degree'].append(G.degree(node))
        metrics['In Degree'].append(G.in_degree(node))
        metrics['Out Degree'].append(G.out_degree(node))

        # Средний вес рёбер для узла
        weights = [G[node][nbr]['weight'] for nbr in G.neighbors(node)]
        metrics['Average Edge Weight'].append(sum(weights) / len(weights) if weights else 0)

    # Преобразуем метрики в DataFrame и отображаем
    df_metrics = pd.DataFrame(metrics)
    display(df_metrics)

    # Создание и отображение таблицы смежности
    adjacency_matrix = nx.to_pandas_adjacency(G, weight='weight')
    display(adjacency_matrix)

# Выполняем анализ и визуализацию графа
analyze_and_visualize_midi_graph(midi_path)
